In [ ]:
import logging, json
from os.path import abspath, dirname, join
from collections import defaultdict
from tqdm.auto import tqdm
from pyserini.search.lucene import LuceneSearcher

import os
import re
import json
import random
from collections import defaultdict
from ast import literal_eval
from decimal import Decimal

logger = logging.getLogger(__name__)


BASE_DIR = "/home/leo/code/webshop"
DEBUG_PROD_SIZE = 1000  # set to `None` to disable

DEFAULT_ATTR_PATH = join(BASE_DIR, "data/items_ins_v2_1000.json")
DEFAULT_FILE_PATH = join(BASE_DIR, "data/items_shuffle_1000.json")
DEFAULT_REVIEW_PATH = join(BASE_DIR, "data/reviews.json")

FEAT_CONV = join(BASE_DIR, "data/feat_conv.pt")
FEAT_IDS = join(BASE_DIR, "data/feat_ids.pt")

HUMAN_ATTR_PATH = join(BASE_DIR, "data/items_human_ins.json")
HUMAN_ATTR_PATH = join(BASE_DIR, "data/items_human_ins.json")

SEARCH_RETURN_N=50

def get_top_n_product_from_keywords(
        keywords,
        search_engine,
        all_products,
        product_item_dict,
        attribute_to_asins=None,
    ):
    if keywords[0] == '<r>':
        top_n_products = random.sample(all_products, k=SEARCH_RETURN_N)
    elif keywords[0] == '<a>':
        attribute = ' '.join(keywords[1:]).strip()
        asins = attribute_to_asins[attribute]
        top_n_products = [p for p in all_products if p['asin'] in asins]
    elif keywords[0] == '<c>':
        category = keywords[1].strip()
        top_n_products = [p for p in all_products if p['category'] == category]
    elif keywords[0] == '<q>':
        query = ' '.join(keywords[1:]).strip()
        top_n_products = [p for p in all_products if p['query'] == query]
    else:
        keywords = ' '.join(keywords)
        hits = search_engine.search(keywords, k=SEARCH_RETURN_N)
        docs = [search_engine.doc(hit.docid) for hit in hits]
        top_n_asins = [json.loads(doc.raw())['id'] for doc in docs]
        top_n_products = [product_item_dict[asin] for asin in top_n_asins if asin in product_item_dict]
    return top_n_products

def generate_product_prices(all_products):
    product_prices = dict()
    for product in all_products:
        asin = product["asin"]
        pricing = product["pricing"]
        if not pricing:
            price = 100.0
        elif len(pricing) == 1:
            price = pricing[0]
        else:
            price = random.uniform(*pricing[:2])
        product_prices[asin] = price
    return product_prices


def init_search_engine(num_products=None):
    if num_products == 100:
        indexes = "indexes_100"
    elif num_products == 1000:
        indexes = "indexes_1k"
    elif num_products == 100000:
        indexes = "indexes_100k"
    elif num_products is None:
        indexes = "indexes"
    else:
        raise NotImplementedError(
            f"num_products being {num_products} is not supported yet."
        )
    search_engine = LuceneSearcher(os.path.join(BASE_DIR, f"search_engine/{indexes}"))
    return search_engine


def clean_product_keys(products):
    for product in products:
        product.pop("product_information", None)
        product.pop("brand", None)
        product.pop("brand_url", None)
        product.pop("list_price", None)
        product.pop("availability_quantity", None)
        product.pop("availability_status", None)
        product.pop("total_reviews", None)
        product.pop("total_answered_questions", None)
        product.pop("seller_id", None)
        product.pop("seller_name", None)
        product.pop("fulfilled_by_amazon", None)
        product.pop("fast_track_message", None)
        product.pop("aplus_present", None)
        product.pop("small_description_old", None)
    print("Keys cleaned.")
    return products


def load_products(filepath, num_products=None, human_goals=True):
    # TODO: move to preprocessing step -> enforce single source of truth
    with open(filepath) as f:
        products = json.load(f)
    print("Products loaded.")
    products = clean_product_keys(products)

    # with open(DEFAULT_REVIEW_PATH) as f:
    #     reviews = json.load(f)
    all_reviews = dict()
    all_ratings = dict()
    # for r in reviews:
    #     all_reviews[r['asin']] = r['reviews']
    #     all_ratings[r['asin']] = r['average_rating']

    if human_goals:
        with open(HUMAN_ATTR_PATH) as f:
            human_attributes = json.load(f)
    with open(DEFAULT_ATTR_PATH) as f:
        attributes = json.load(f)
    with open(HUMAN_ATTR_PATH) as f:
        human_attributes = json.load(f)
    print("Attributes loaded.")

    asins = set()
    all_products = []
    attribute_to_asins = defaultdict(set)
    if num_products is not None:
        # using item_shuffle.json, we assume products already shuffled
        products = products[:num_products]
    for i, p in tqdm(enumerate(products), total=len(products)):
        asin = p["asin"]
        if asin == "nan" or len(asin) > 10:
            continue

        if asin in asins:
            continue
        else:
            asins.add(asin)

        products[i]["category"] = p["category"]
        products[i]["query"] = p["query"]
        products[i]["product_category"] = p["product_category"]

        products[i]["Title"] = p["name"]
        products[i]["Description"] = p["full_description"]
        products[i]["Reviews"] = all_reviews.get(asin, [])
        products[i]["Rating"] = all_ratings.get(asin, "N.A.")
        for r in products[i]["Reviews"]:
            if "score" not in r:
                r["score"] = r.pop("stars")
            if "review" not in r:
                r["body"] = ""
            else:
                r["body"] = r.pop("review")
        products[i]["BulletPoints"] = (
            p["small_description"]
            if isinstance(p["small_description"], list)
            else [p["small_description"]]
        )

        pricing = p.get("pricing")
        if pricing is None or not pricing:
            pricing = [100.0]
            price_tag = "$100.0"
        else:
            pricing = [
                float(Decimal(re.sub(r"[^\d.]", "", price)))
                for price in pricing.split("$")[1:]
            ]
            if len(pricing) == 1:
                price_tag = f"${pricing[0]}"
            else:
                price_tag = f"${pricing[0]} to ${pricing[1]}"
                pricing = pricing[:2]
        products[i]["pricing"] = pricing
        products[i]["Price"] = price_tag

        options = dict()
        customization_options = p["customization_options"]
        option_to_image = dict()
        if customization_options:
            for option_name, option_contents in customization_options.items():
                if option_contents is None:
                    continue
                option_name = option_name.lower()

                option_values = []
                for option_content in option_contents:
                    option_value = (
                        option_content["value"].strip().replace("/", " | ").lower()
                    )
                    option_image = option_content.get("image", None)

                    option_values.append(option_value)
                    option_to_image[option_value] = option_image
                options[option_name] = option_values
        products[i]["options"] = options
        products[i]["option_to_image"] = option_to_image

        # without color, size, price, availability
        # if asin in attributes and 'attributes' in attributes[asin]:
        #     products[i]['Attributes'] = attributes[asin]['attributes']
        # else:
        #     products[i]['Attributes'] = ['DUMMY_ATTR']
        # products[i]['instruction_text'] = \
        #     attributes[asin].get('instruction', None)
        # products[i]['instruction_attributes'] = \
        #     attributes[asin].get('instruction_attributes', None)

        # without color, size, price, availability
        if asin in attributes and "attributes" in attributes[asin]:
            products[i]["Attributes"] = attributes[asin]["attributes"]
        else:
            products[i]["Attributes"] = ["DUMMY_ATTR"]

        if human_goals:
            if asin in human_attributes:
                products[i]["instructions"] = human_attributes[asin]
        else:
            products[i]["instruction_text"] = attributes[asin].get("instruction", None)

            products[i]["instruction_attributes"] = attributes[asin].get(
                "instruction_attributes", None
            )

        products[i]["MainImage"] = p["images"][0]
        products[i]["query"] = p["query"].lower().strip()

        all_products.append(products[i])

    for p in all_products:
        for a in p["Attributes"]:
            attribute_to_asins[a].add(p["asin"])

    product_item_dict = {p["asin"]: p for p in all_products}
    product_prices = generate_product_prices(all_products)
    return all_products, product_item_dict, product_prices, attribute_to_asins


class Webshop:
    def __init__(self):
        self.products = []
        self.page_name = "home"
        self.page_params = {}
        self.clickables = {}
        self.inputs = {}
        self.page = ""
        self.page_stack = []
        self.page_inited = False
        self.render()

        self.all_products, self.product_item_dict, self.product_prices, _ = (
            load_products(
                filepath=DEFAULT_FILE_PATH, num_products=None, human_goals=True
            )
        )
        self.search_engine = init_search_engine(num_products=1000)
        self.purchase_callback = lambda product, options: print(f"Purchased {product['Title']} with options {options}")

    def type(self, name, text):
        if name in self.inputs:
            self.inputs[name] = text
            self.render()
        else:
            logger.error(f"INVALID ACTION: Input field {name} not found")

    def click(self, name):
        if name in self.clickables:
            self.clickables[name]()
            self.render()
        else:
            logger.error(f"INVALID ACTION: Clickable {name} not found")

    def render(self):
        if self.page_name == "home":
            if not self.page_inited:
                self.render_homepage(init=True)
                self.page_inited = True
            self.render_homepage()
        elif self.page_name == "search":
            if not self.page_inited:
                self.render_search(init=True)
                self.page_inited = True
            self.render_search()
        elif self.page_name == "product":
            if not self.page_inited:
                self.render_product(init=True)
                self.page_inited = True
            self.render_product()

    def push_stack(self):
        self.page_stack.append(
            {
                "page_name": self.page_name,
                "page_params": self.page_params,
            }
        )

    def pop_stack(self):
        if len(self.page_stack) > 0:
            self.page_name = self.page_stack[-1]["page_name"]
            self.page_params = self.page_stack[-1]["page_params"]
            self.page_stack.pop()
        else:
            self.page_name = "home"
            self.page_params = {}
        self.page_inited = False
        self.render()

    def go(self, page_name, page_params):
        self.push_stack()
        self.page_name = page_name
        self.page_params = page_params
        self.page_inited = False

    def render_homepage(self, init=False):
        if init:
            self.clickables = {
                "search": lambda: self.go("search", {"search_term": self.inputs["search"]})
            }
            self.inputs = {"search": ""}
            self.page_params = {}
        self.page = f"""
<input type="text" name="search" placeholder="Search" value="{self.inputs['search']}">
<button type="submit" name="search">Search</button>
"""

    def render_search(self, init=False):
        if init:
            self.page_params = {
                'search_term': self.page_params['search_term'],
                'page': 0 if 'page' not in self.page_params else self.page_params['page'],
            }
            self.inputs = {
                'searchbox.search': self.page_params['search_term'],
            }
        top_n_products = get_top_n_product_from_keywords(
            self.page_params['search_term'].split(),
            self.search_engine,
            self.all_products,
            self.product_item_dict,
        )
        products = top_n_products[self.page_params['page']*10:(self.page_params['page']+1)*10]
        n_pages = len(top_n_products) // 10
        
        products_html = [
            f"""
            <div class="product" name="{p['asin']}">
                <img src="{p['MainImage']}" alt="{p['Title']}">
                <div class="product-info">
                    <h3>{p['Title']}</h3>
                    <p>{p['Description']}</p>
                    <p>{p['Price']}</p>
                    <button type="submit" name="view">View</button>
                </div>
            </div>
            """
            for p in products
        ]
        pagination_html = [
            f"""
            <button type="submit" name="{i+1}">{i+1}</button>
            """
            for i in range(n_pages)
        ]
        pagination_html = [
            f"""
            <button type="submit" name="back">Back</button>
            """ if self.page_params['page'] > 0 else ""
        ] + pagination_html + [
            f"""
            <button type="submit" name="next">Next</button>
            """ if self.page_params['page'] < n_pages-1 else ""
        ]
        
        self.page = f"""
<div name="searchbox">
<input type="text" name="search" placeholder="Search" value="{self.inputs['searchbox.search']}">
<button type="submit" name="search">Search</button>
</div>
<div name="searchresult">
{''.join(products_html)}
</div>
<div name="pagination">
viewing page {self.page_params['page']+1} of {n_pages}
{''.join(pagination_html)}
</div>
"""
        # self.clickables = {"searchbox.search": lambda: self.go("search", {"search_term": self.inputs["search"]})}
        self.clickables = {
            "searchbox.search": lambda: self.go("search", {"search_term": self.inputs["search"]}),
            **{
                f"searchresult.{p['asin']}.view": lambda asin=p['asin']: self.go("product", {"asin": asin})
                for p in products
            },
            "pagination.back": lambda: self.go("search", {"search_term": self.page_params["search_term"], "page": self.page_params["page"]-1}),
            "pagination.next": lambda: self.go("search", {"search_term": self.page_params["search_term"], "page": self.page_params["page"]+1}),
            **{
                f"pagination.{i}": lambda: self.go("search", {"search_term": self.page_params["search_term"], "page": i-1})
                for i in range(n_pages)
            }
        }

    def render_product(self, init=False):
        print(self.page_params)
        if init:
            product = self.product_item_dict[self.page_params['asin']]
            self.page_params = {
                'asin': self.page_params['asin'],
                'options': {option_name: product['options'][option_name][0] for option_name in product['options']},
            }
            self.inputs = {}
        product = self.product_item_dict[self.page_params['asin']]
        options_html = [
            f"""
            <div class="option" name="{option_name}">
                <h4>{option_name}</h4>
                <ul>
                    {''.join([f'<button name="{option_value}">{option_value} {"(selected)" if self.page_params["options"][option_name] == option_value else ""}</button>' for option_value in option_values])}
                </ul>
            </div>
            """
            for option_name, option_values in product['options'].items()
        ]
        self.page = f"""
<div name="product">
    <img src="{product['MainImage']}" alt="{product['Title']}">
    <div>
        <h3>{product['Title']}</h3>
        <p>{product['Price']}</p>
        <p>{product['Description']}</p>
        <p>{product['BulletPoints']}</p>
        <button type="submit" name="purchase">Add to Cart</button>
    </div>
</div>
<div class="options">
{''.join(options_html)}
</div>
"""
        def select_option(option_name, option_value):
            self.page_params['options'][option_name] = option_value
        self.clickables = {
            "product.purchase": lambda: self.purchase_callback(product, self.page_params['options']),
            **{
                f"options.{option_name}.{option_value}": lambda option_name=option_name, option_value=option_value: select_option(option_name, option_value)
                for option_name, option_values in product['options'].items()
                for option_value in option_values
            }
        }
        

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''
from simulated_web_agent.webshop.env import Webshop
w = Webshop()

In [ ]:
print(w.page)

In [ ]:
w.type("search", "woman faux fur jacket")

In [ ]:
w.click("search")

In [ ]:
w.clickables

In [ ]:
print(w.page)

In [ ]:
w.click("searchresult.B09KP78G37.view")

In [ ]:
print(w.page)

In [ ]:
w.click("options.size.medium")

In [ ]:
print(w.page)

In [ ]:
w.page_stack

In [ ]:
w.click('product.purchase')

In [ ]:
w.product_item_dict['B09KP78G37']

# Agent

In [1]:
import os, json
os.environ['OPENAI_API_KEY'] = ''
from simulated_web_agent.webshop.env import WebshopEnv
from simulated_web_agent.agent.gpt import chat, chat_bulk
e = WebshopEnv()
env = e.reset()
import nest_asyncio
nest_asyncio.apply()

Products loaded.
Keys cleaned.
Attributes loaded.


  0%|          | 0/1000 [00:00<?, ?it/s]

In [2]:
# e.step(r"""{"type":"type","name":"search","text":"woman faux jacket"}""")

In [3]:
# env = e.step(r"""{"type":"click","name":"search"}""")[0]

In [4]:
# print(env['page'])

In [5]:
from simulated_web_agent.agent import Agent
from simulated_web_agent.agent.gpt import chat, embed_text

In [24]:
agent = Agent("""
Persona: Clara
Background:
Clara is a PhD student in Computer Science at a prestigious university. She is deeply engaged in research focusing on artificial intelligence and machine learning, aiming to contribute to advancements in technology that can benefit society.

Demographics:

Age: 28
Gender: Female
Education: Pursuing a PhD in Computer Science
Professional Life:
Clara spends most of her time in academia, attending conferences, working in the lab, and writing papers. Her commitment to her research is her main priority, and she manages her time around her academic responsibilities.

Financial Situation:
Clara have a rich family, so she cares quality of the goods more than the price.


Shopping Habits:
Clara dislikes shopping and avoids spending much time browsing through products. She prefers straightforward, efficient shopping experiences and often shops online for convenience. When she does shop, she looks for practicality and affordability over style or trendiness.

Personal Style:
Clara prefers comfortable, functional clothing, often choosing items that are easy to wear for long hours spent at her desk or in the lab. She wears XXL-sized clothing and likes colors that reflect her personality—mostly grey, which she finds uplifting and energizing.
""", "buy a jacket")

In [25]:
agent.perceive(json.dumps(env))

In [17]:
agent.memory.update()

In [18]:
agent.memory.memories[-1].content

"A button with the name 'search' and the text 'Search'."

In [19]:
agent.reflect()

In [20]:
agent.plan()

In [21]:
print(agent.current_plan.content)

1. (next) Search for 'XXL grey woman jacket'
2. Browse search results to know the general price range, quality, and availability of jackets.
3. Browse into specific product for more details
    1. Browse into product with name 'XXX' and ASIN 'XXX', check the price, color, size, quality, and description
    2. Browse into product with name 'AAA' and ASIN 'BBB', check the price, color, size, quality, and description


In [22]:
action = agent.act(env)
action

{'type': 'click',
 'name': 'search',
 'description': "Click the search button to perform the search for 'XXL grey woman jacket'."}

In [23]:
env, _, _, _, _ = e.step(json.dumps(action))
env

search result ['B07FD13LP1', 'B08DXL22JN', 'B09KP78G37', 'B098XT346Y', 'B07DKGJR74', 'B09R7H66FC', 'B09GLVMLMS', 'B09QPX97VW', 'B01N1T77OM', 'B001B4NFV0']
search result ['B07FD13LP1', 'B08DXL22JN', 'B09KP78G37', 'B098XT346Y', 'B07DKGJR74', 'B09R7H66FC', 'B09GLVMLMS', 'B09QPX97VW', 'B01N1T77OM', 'B001B4NFV0']


{'url': 'http://127.0.0.1/search?search_term=XXL+grey+woman+jacket&page=0',
 'page': '\n<div name="searchbox">\n<input type="text" name="searchbox.search" placeholder="Search" value="XXL grey woman jacket">\n<button type="submit" name="search">Search</button>\n</div>\n<div name="searchresult">\n\n            <div class="product" name="B07FD13LP1">\n                <img src="https://m.media-amazon.com/images/I/31gQUsTMfgS.jpg" alt="The Establishment Men\'s Organic Cotton Fleece Short with Pocket(S to XXL)">\n                <div class="product-info">\n                    <h3>The Establishment Men\'s Organic Cotton Fleece Short with Pocket(S to XXL)</h3>\n                    <p></p>\n                    <p>$24.99</p>\n                    <button type="submit" name="searchresult.B07FD13LP1.view">View</button>\n                </div>\n            </div>\n            \n            <div class="product" name="B08DXL22JN">\n                <img src="https://m.media-amazon.com/images/I/41UXlJTj

In [ ]:
# we reflect on the most recent memories
# two most recent memories (last observasion, reflect, plan, action)
memories = [
    i for i in agent.memory.memories if i.timestamp >= agent.memory.timestamp - 1
]
memories = [
    f"""timestamp: {m.timestamp}; kind: {m.kind}; content: {m.content}"""
    for m in memories
]

In [ ]:
prompt = """
You are tasked with reflecting on a given set of observations from a simulated web page interaction. The observations include various elements on the page, such as search boxes, buttons, product listings, images, and pagination.

Your goal is to generate insightful questions based on these observations to facilitate a deeper understanding and analysis. 

Examples of questions to consider:
1. What is the price range of the search results?
2. What is the average price of the items listed?
3. How many products are displayed on the page?
4. Are there any patterns in the product names or categories?
5. How is the search functionality implemented based on the observed elements?
6. What types of products are most common in the search results?
7. Are there any notable differences in the presentation of different products?
8. How is pagination handled, and what does it indicate about the total number of results?

Your output should be a list of questions formatted as follows:

```json
{
    "questions": [
        "<question 1>",
        "<question 2>",
        ...
    ]
}
```
"""

In [ ]:
reflect = chat([{
    'role': 'system',
    'content': prompt
},
      {
            'role': 'user',
            'content': json.dumps(memories)
      }],
            response_format={"type": "json_object"},
               
               )

In [ ]:
print(reflect.choices[0].message.content)

In [ ]:
memories = agent.memory.retrieve( "What is the range of prices for the jackets listed in the search results?")
memories = [
    f"""index: {i}, timestamp: {m.timestamp}; kind: {m.kind}; content: {m.content}"""
    for i, m in enumerate(memories)
]

In [ ]:
prompt = """
You are tasked with reflecting on a specific question based on a set of observations from a simulated web page interaction. You will be provided with a single question and a list of relevant memories (with index, timestamp, kind and content) pertaining to that question.

Your goal is to answer the question using the information available in the memories. If the question cannot be answered based on the given memories, output "N/A".

The answer should be a complete sentence with all the information in the question, for example, "The range of prices for the jackets listed in the search results is $50 to $100." instead of just "$50 to $100".

You should associate the answer with the relevant memories by providing their indices in the memories list.
You should consider the current timestamp and the timestamps of the memories to determine the most relevant information for answering the question.

Format your output as follows:

```json
{
    "answer": "<your answer or 'N/A'>",
    "target": ["<target_index_1>", "<target_index_2>"]
}
```
"""

In [ ]:
answer = chat([{
    'role': 'system',
    'content': prompt
},
      {
            'role': 'user',
            'content': json.dumps({
                "question": "What is the range of prices for the jackets listed in the search results?",
                "memories": memories
            })
      }],
            response_format={"type": "json_object"},
               
               )

In [ ]:
print(answer.choices[0].message.content)

In [ ]:
memories

In [ ]:
reflections = [i for i in agent.memory.memories if i.kind == 'reflection']

In [ ]:
prompt = """You are tasked with evaluating the importance of a given fact in helping to achieve a specified intent from the perspective of a provided persona.

You will be given:
- An fact
- A persona
- An intent

Your goal is to assess how crucial the fact is for fulfilling the intent from the persona's point of view.

Output a score from 1 to 10, where 1 indicates the fact is not important at all and 10 indicates the fact is extremely important.

Format your output as a JSON:

```json
{
    "importance_score": <score from 1-10>
}
```"""

In [ ]:
reflections

In [ ]:
requests = [
    [
        {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": json.dumps({
                "answer": r.content,
                "persona": agent.persona,
                "intent": agent.intent,
            }),
        }
    ]
    for r in reflections
]
import openai, asyncio
client = openai.Client()
async_client = openai.AsyncClient()
embedding_model = "text-embedding-3-small"
chat_model = "gpt-4o-mini"

async def chat_bulk(messages, model=chat_model, **kwargs):
    async def chat_one(message):
        return await async_client.chat.completions.create(
            model=model, messages=message, **kwargs
        )
    return await asyncio.gather(*[chat_one(m) for m in messages])
responses = await chat_bulk(requests, response_format={"type": "json_object"})
# print(requests[0])
# response = chat(requests[0], response_format={"type": "json_object"})

In [ ]:
responses

In [ ]:
[r.content for r in reflections]

In [ ]:
r = agent.memory.memories[-20:0]

In [ ]:
for i in range(-7, 0):
    print(i)
    print(i, agent.memory.memories[i].content)
    
    print(i, agent.memory.memories[i].importance)

In [ ]:
for i in range(0,-20,-1):
    print(i)
    print(i, agent.memory.memories[i].content)
    
    # print(i, agent.memory.memories[i].importance)

In [ ]:
prompt = """You are tasked with creating a detailed plan based on the given persona, intent, and relevant memories (including observations, reflections, and previous actions if any).

Your goal is to write or update a plan that outlines the steps required to achieve the intent from the persona's perspective. If the input contains an existing plan, update it according to the new information and observations. If there is no existing plan, create a new one from scratch.

The previous plan, if any, is from the last step.

The plan should be structured in a tree-like format, detailing each step and any sub-steps necessary to accomplish the intent. Always adhere to the preferences and characteristics of the persona (e.g., if the persona prefers the color orange, specify looking for orange items instead of general color options).

Think in two steps:
1. Write or update the plan. Do not rewrite any plan before the (next) step, but you can rewrite the plan after the (next) step. Revise the plan and make necessary changes based on the new information and observations. For example, if the original plan said "browse search results" and in your observation there are search results listed, you should write many sub-plans like "look into product with ASIN xxxx".
2. Determine which step or sub-step is (next). Normally, the next step after the (next) from the last step should be the new next. You should always move the (next) marker. You should always move the (next) marker. You should always move the (next) marker. You should always move the (next) marker. Ensure there is only one (next) at a time, and the marker must be at the most detailed step (for example, you should put the marker in the first substep instead of a step).


You will also be given a current timestamp. All memories with the same timestamp are current. Older but relevant memories will be provided for context.

Format your output as follows:

```json
{
    "plan": "<string representation of the plan>"
}
```

Examples of plans:
1. search for "woman jacket"
2. (next) browse search results to know the general price range of jackets, and determine if that meets my budget.
3. browse into specific product for more details
    1. browse into product with name "XXX" and ASIN "XXX", check the price, color, size and description
    2. browse into product with name "AAA" and ASIN "BBB", check the price, color, size and description

Always stick to the persona's preferences and characteristics."""

In [ ]:
print()